# Data Preparation for the 2022/23 Birmingham Lions statistics

### Foreword

I have spent around 2 weeks watching every snap, kick and dead play of football that the Birmingham Lions played this season and have decided to collect the stats for the season. This, admittedly, is the byproduct of a larger mission I have to increase the use of data in the Birmingham Lions organisation, so I can spearhead a move towards data backed decisions. American Football is one of the most data driven sports in the world and I believe our path to success must begin with us embracing this part of the game and becoming the best around at collating and utilising this data. 'Built not Bought' is what we believing in Birmingham, and I believe this is how we can start to build an even better product - a winning one.

### Current state of the data

I have used Excel spreadsheets to keep track of the data, collating the stats accrued by a player on a play by play basis. I had a template for each Defensive, Rushing, Passing and Returning attempt but often I found that the requirements for each game changed from game to game. For example (spoiler incoming), at UEA Home we scored a defensive touchdown, but this didn't occur in any other game. This led to UEA away having a defensive touchdown column but no other defensive table has one. This isn't immediately a problem as aggregations and grouping can happen can happen on each individual table but when you want to create a union between all of the defensive tables, this isn't great. In this notebook, I will be flexing my Python muscles and fixing the inconsistencies which exist in the passing data tables.

##### Passing Data

None of us came here to read me talk, but instead are either here to enjoy some American Football analysis or judge my programming abilities, either of which I am fine with - I just wanted to address the elephants in the room. When converting the data from spreadsheets into csv files, I noticed that the QB Data had 3 main formats, which isn't condusive of a union between the tables.

In [1]:
import pandas as pd

UEAA= pd.read_csv("UEA_A_Q.csv") 
FILA=pd.read_csv("FIL_A_Q.csv") 
FILH=pd.read_csv("FIL_H_Q.csv") 
CCH=pd.read_csv("CC_H_Q.csv") 
UWEA=pd.read_csv("UWE_A_Q.csv") 
UWEH=pd.read_csv("UWE_H_Q.csv") 
CCA=pd.read_csv("CC_A_Q.csv") 
UEAH=pd.read_csv("UEA_H_Q.csv") 
LEEA=pd.read_csv("LEE_A_Q.csv") 

In [2]:
print(UEAA.head())
print(CCH.head())
print(LEEA.head())

   Play  Player Completion  Yards Pressure  Target   TD  Int Sack  QBFum  \
0     6      10          N    NaN        Y     8.0  NaN  NaN  NaN    NaN   
1     7      10          N    NaN        Y    11.0  NaN  NaN  NaN    NaN   
2    20      10          N    NaN        Y     8.0  NaN  NaN  NaN    NaN   
3    21      10          N    NaN        N     8.0  NaN  NaN  NaN    NaN   
4    22      10          N    NaN        N    49.0  NaN  NaN  NaN    NaN   

   FumRec  RecFum  FumYards  
0     NaN     NaN       NaN  
1     NaN     NaN       NaN  
2     NaN     NaN       NaN  
3     NaN     NaN       NaN  
4     NaN     NaN       NaN  
   Play  Player Completion  Yards Pressure  Target  TD  Int  Sack  QBFum  \
0     2       9          N    NaN        Y      85 NaN  1.0   NaN    NaN   
1    17       9          N    NaN        N      85 NaN  NaN   NaN    NaN   
2    24       9          Y    2.0        N       8 NaN  NaN   NaN    NaN   
3    30       9          N    NaN        N       8 NaN  NaN

As you can see, there are different columns for each of these Passing tables. The first problem is that the Fumble column ('Fum') has been labelled as ('QBFum') for the Cardiff Home game. Although QBFum is the outlier, I believe that all tables should have the QBFum, RecFum labelling as it is clear who made the fumble on the play. From my recollection, I known that all fumbles in the Fum table are by the Quarterback and thus, count as QBFums. We will be renaming each of the Fum columns to QBFum.

In [3]:
renameGames=[UEAA, FILA, FILH, CCA, UEAH, LEEA]
for game in renameGames:
    game.rename(columns={'Fum':'QBFum'}, inplace=True)

We can check this has worked by printing one of the columns of the previously incorrect tables.

In [4]:
FILA.head()

,Play,Player,Completion,Yards,Pressure,Target,TD,Int,Sack,QBFum,FumRec,RecFum,FumYards
0,4,10,N,,N,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,10,Y,3,N,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,24,10,S,NaN,Y,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
3,25,10,Y,8,N,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,26,10,N,NaN,Y,11.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN


We can see this worked correctly. Another problem is that UEA Away didn't actually have a fumble column so we can add that in, filling it with a NaN value as there were no fumbles recorded. Also, all games need a FumRec column like LEEA has.

In [5]:
import numpy as np

UEAA['QBFum'] = np.nan

FumRecGames=[UEAA, FILA, FILH, CCH, UWEA, UWEH, CCA, UEAH]
for game in FumRecGames:
    game['FumRec'] = np.nan

In [6]:
UEAA.head()

,Play,Player,Completion,Yards,Pressure,Target,TD,Int,Sack,QBFum,FumRec,RecFum,FumYards
0,6,10,N,NaN,Y,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,10,N,NaN,Y,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20,10,N,NaN,Y,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21,10,N,NaN,N,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,10,N,NaN,N,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Comparing this table with the Filton away table, we can see that they have the same columns and this is good.

Another problem with this same list of passing tables is that they didn't have the columns RecFum and FumYards, which are columns I figured out that I needed once the first receiver fumble after the catch happened against UWE away. There were no RecFums or FumYards in the rest of these games so we can fill these with NaN, similar to how we did with UEA away's Fumble column.

In [7]:
for game in renameGames:
    game['RecFum']=np.nan
    game['FumYards']=np.nan

In [8]:
UEAA.head()

,Play,Player,Completion,Yards,Pressure,Target,TD,Int,Sack,QBFum,FumRec,RecFum,FumYards
0,6,10,N,NaN,Y,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,10,N,NaN,Y,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20,10,N,NaN,Y,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21,10,N,NaN,N,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,22,10,N,NaN,N,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
UWEA.head()

,Play,Player,Completion,Yards,Pressure,Target,TD,Int,Sack,QBFum,RecFum,FumYards,FumRec
0,8,10,N,NaN,Y,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,10,N,NaN,N,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,10,N,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23,10,Y,24.0,Y,8.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,34,10,Y,1.0,Y,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
LEEA.head()

,Play,Player,Completion,Yards,Pressure,Target,TD,Int,Sack,QBFum,FumRec,RecFum,FumYards
0,21,12,Y,10.0,Y,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22,12,N,NaN,Y,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25,12,N,NaN,Y,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27,12,S,-10.0,Y,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,38,12,N,NaN,Y,8.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN


At this point, we can see that all passing tables have the same columns, which is exactly what we wanted. These are all the changes that were required to be made to passing data so we will overwrite the files that we initially extracted from.

In [11]:
UEAA.to_csv("UEA_A_Q.csv", index=False)
FILA.to_csv("FIL_A_Q.csv", index=False)
FILH.to_csv("FIL_H_Q.csv", index=False)
CCH.to_csv("CC_H_Q.csv", index=False)
UWEA.to_csv("UWE_A_Q.csv", index=False)
UWEH.to_csv("UWE_H_Q.csv", index=False)
CCA.to_csv("CC_A_Q.csv", index=False)
UEAH.to_csv("UEA_H_Q.csv", index=False)
LEEA.to_csv("LEE_A_Q.csv", index=False)

##### Defensive Data

I will give myself a pat on the back for having most of the other tables completely correct across all of the data. The only change that needs to occur now is that a touchdown column needs to be added to all defensive columns, except for the UEA home game.

In [12]:
UEAA_D= pd.read_csv("UEA_A_D.csv") 
FILA_D=pd.read_csv("FIL_A_D.csv") 
FILH_D=pd.read_csv("FIL_H_D.csv") 
CCH_D=pd.read_csv("CC_H_D.csv") 
UWEA_D=pd.read_csv("UWE_A_D.csv") 
UWEH_D=pd.read_csv("UWE_H_D.csv") 
CCA_D=pd.read_csv("CC_A_D.csv") 
UEAH_D=pd.read_csv("UEA_H_D.csv") 
LEEA_D=pd.read_csv("LEE_A_D.csv") 

In [13]:
UEAH_D.head()

,Play,Player,Tackle,TFL,Pressure,Sack,Int,LOC,FF,FR,TD
0,7,32,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7,93,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,40,H,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8,20,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9,5,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here we can see the TD column and verify it's name.

In [14]:
TdGames=[UEAA_D, FILA_D, FILH_D, CCH_D, UWEA_D, UWEH_D, CCA_D, LEEA_D]

for game in TdGames:
    game['TD'] = np.nan

In [15]:
UEAA_D.head()

,Play,Player,Tackle,TFL,Pressure,Sack,Int,LOC,FF,FR,TD
0,9,51,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,9,40,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10,54,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,37,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,32,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
UEAA_D.to_csv("UEA_A_D.csv", index=False)
FILA_D.to_csv("FIL_A_D.csv", index=False)
FILH_D.to_csv("FIL_H_D.csv", index=False)
CCH_D.to_csv("CC_H_D.csv", index=False)
UWEA_D.to_csv("UWE_A_D.csv", index=False)
UWEH_D.to_csv("UWE_H_D.csv", index=False)
CCA_D.to_csv("CC_A_D.csv", index=False)
UEAH_D.to_csv("UEA_H_D.csv", index=False)
LEEA_D.to_csv("LEE_A_D.csv", index=False)

Assuming all went correctly, this should be enough to start the analysis of this data. Thanks for reading!

Now onto the exciting bit... until I remmebereed that I wanted a table of players.

##### Creation of players table

In [17]:
UEAA=pd.read_csv("UEAA.csv") 
FILA=pd.read_csv("FILA.csv") 
FILH=pd.read_csv("FILH.csv") 
CCH=pd.read_csv("CCH.csv") 
UWEA=pd.read_csv("UWEA.csv") 
UWEH=pd.read_csv("UWEH.csv") 
CCA=pd.read_csv("CCA.csv") 
UEAH=pd.read_csv("UEAH.csv") 
LEEA=pd.read_csv("LEEA.csv") 

In [18]:
games=[UEAA, FILA, FILH, CCH, UWEA, UWEH, CCA, UEAH, LEEA]

In [19]:
long_list= pd.concat(games)

In [20]:
long_list.tail()

,Number,Name
44,86,Nick Huneidi
45,88,Dan Bruton
46,90,Damilola Iyun
47,93,Bhavya Gandhi
48,99,Niklas Adio


In [21]:
short_list = np.sort(long_list.Name.unique())

In [22]:
short_list

array(['Aaron Mante', 'Adam Chouder', "Adam O'Reilly", 'Ahmed Alamin',
       'Ahmed Alamin ', 'Alex Baer', 'Amrit Dhesi', 'Andrew Saffar',
       'Ben El Hawary', 'Ben El-Hawary', 'Ben Flint', 'Bhavya Gandhi',
       'Brennan Dyball', 'Bryan Okai', 'Chris Kasa', 'Christan Covi',
       'Cormac Connell', 'Damilola Iyun', 'Dan Bruton', 'Dan Wickes',
       'David Ononye', 'Demi Agienoji', 'Ed Cohen', 'Eddie Perry',
       'Eliakim Alem', 'Elijah Balch', 'Elijah Okafor',
       'Ella Shakespeare', 'Emily Armitage', 'Emily Irvine',
       'Ethan Eggleston', 'Ethan Payne', 'Frazer Shaw', 'Freddie James',
       'Gilead Bempah', 'Gilean Bempah', 'Grady Nightingale',
       'Hung Yeh Chien', 'Hung Yuh Chien', 'Ivan Abbott',
       'James Schofield', 'James Williams', 'Jeevan Singh Soomal',
       'Jeevan Soomal', 'Jesse Onyas', 'Jesse Otah', 'Jip De Vries',
       'Joey Williamson', 'Johnny Hakim', 'Kevin Davies',
       'Khalil Ibrahim', 'Kyle Jay Insley', 'Laurence Piper',
       'Lewis Wa

In [23]:
players = pd.DataFrame({
    'id':range(1, short_list.size+1),
    'Name':short_list
})

In [24]:
players.head()

,id,Name
0,1,Aaron Mante
1,2,Adam Chouder
2,3,Adam O'Reilly
3,4,Ahmed Alamin
4,5,Ahmed Alamin


In [25]:
mistakes = {
    'Ahmed Alamin ':'Ahmed Alamin',
    'Ben El Hawary':'Ben El-Hawary',
    'Gilean Bempah':'Gilead Bempah',
    'Hung Yuh Chien':'Hung Yeh Chien',
    'Jeevan Soomal':'Jeevan Singh Soomal',
    'Lewis Wa Tok Tsong':'Lewis Wa Lok Tsong',
    'Rhythm Shreshtha':'Rhythm Shrestha',
    'Tovy Rubenstein':'Tovy Rubinstein',
           }

In [26]:
UEAA=UEAA.merge(players).rename(columns={'id':'playerID'})
FILA=FILA.merge(players).rename(columns={'id':'playerID'})
FILH=FILH.merge(players).rename(columns={'id':'playerID'})
CCH=CCH.merge(players).rename(columns={'id':'playerID'})
UWEA=UWEA.merge(players).rename(columns={'id':'playerID'})
UWEH=UWEH.merge(players).rename(columns={'id':'playerID'})
CCA=CCA.merge(players).rename(columns={'id':'playerID'})
UEAH=UEAH.merge(players).rename(columns={'id':'playerID'})
LEEA=LEEA.merge(players).rename(columns={'id':'playerID'})

In [27]:
UWEH

,Number,Name,playerID
0,5,Aaron Mante,1
1,8,James Schofield,41
2,10,Noah Murray,66
3,15,Ethan Eggleston,31
4,16,Tom Richardson,84
5,19,Demi Agienoji,22
6,20,James Williams,42
7,24,Laurence Piper,53
8,25,Milo Wakefield,60
9,28,Christan Covi,16


In [28]:
mistakes = {
    'Ahmed Alamin ':'Ahmed Alamin',
    'Ben El Hawary':'Ben El-Hawary',
    'Gilean Bempah':'Gilead Bempah',
    'Hung Yuh Chien':'Hung Yeh Chien',
    'Jeevan Soomal':'Jeevan Singh Soomal',
    'Lewis Wa Tok Tsong':'Lewis Wa Lok Tsong',
    'Rhythm Shreshtha':'Rhythm Shrestha',
    'Tovy Rubenstein':'Tovy Rubinstein',
           }

In [29]:
games=[UEAA, FILA, FILH, CCH, UWEA, UWEH, CCA, UEAH, LEEA]
for game in games:
    game['playerID'] = game.Name.apply(lambda x: players[players.Name == mistakes[x]].id.values[0] if x in mistakes.keys() else players[players.Name==x].id.values[0])

In [30]:
games=[UEAA, FILA, FILH, CCH, UWEA, UWEH, CCA, UEAH, LEEA]
a = pd.concat(games)

In [37]:
players[players.Name =='Ben El Hawary']

,id,Name
8,9,Ben El Hawary


In [38]:
a[a.playerID == 9]

,Number,Name,playerID


In [35]:
UEAA=UEAA.drop(columns="Name")
FILA=FILA.drop(columns="Name")
FILH=FILH.drop(columns="Name")
CCH=CCH.drop(columns="Name")
UWEA=UWEA.drop(columns="Name")
UWEH=UWEH.drop(columns="Name")
CCA=CCA.drop(columns="Name")
UEAH=UEAH.drop(columns="Name")
LEEA=LEEA.drop(columns="Name")

In [36]:
UEAA

,Number,playerID
0,5,1
1,8,41
2,10,66
3,11,69
4,12,32
5,14,30
6,15,31
7,16,84
8,18,86
9,19,81


In [39]:
players

,id,Name
0,1,Aaron Mante
1,2,Adam Chouder
2,3,Adam O'Reilly
3,4,Ahmed Alamin
4,5,Ahmed Alamin
...,...,...
85,86,Tovy Rubinstein
86,87,Vojtech Ryp
87,88,Zack Hermon
88,89,Zak Grahame


In [53]:
UEAA.to_csv("UEAA.csv", index=False) 
FILA.to_csv("FILA.csv", index=False)
FILH.to_csv("FILH.csv", index=False)
CCH.to_csv("CCH.csv", index=False)
UWEA.to_csv("UWEA.csv", index=False)
UWEH.to_csv("UWEH.csv", index=False)
CCA.to_csv("CCA.csv", index=False)
UEAH.to_csv("UEAH.csv", index=False)
LEEA.to_csv("LEEA.csv", index=False)

In [54]:
players.to_csv("Players.csv", index=False)